In [1]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
from os import listdir

path_lab = 'C:\\Users\\USER\\Desktop\\후지 0단계\\labels\\' ## 라벨 경로
path_img = 'C:\\Users\\USER\\Desktop\\후지 0단계\\images\\' ## 사진 경로

In [2]:
# ia.seed(1)
def readBox(idx, filename):
    with open(path_lab + 'original\\' + filename, 'r') as f:  ## 라벨링 데이터 불러오기
        boundingboxes = f.read()
        
    boundingboxes = boundingboxes.replace('\n', '')
    
    boundingboxif = boundingboxes.split(" ")   
    
    for j in range(len(boundingboxif)):
        if(j == 0): 
            boundingboxif[0] = int(boundingboxif[0])
        else:
            boundingboxif[j] = float(boundingboxif[j])
            
    return boundingboxif;            

In [3]:
## YOLO text파일의 상대좌표 -> 절대좌표로 변환
def change(bbox, size_original):
    x1 = (bbox[1] - bbox[3]/2) * size_original[1]
    x2 = (bbox[1] + bbox[3]/2) * size_original[1]
    y1 = (bbox[2] - bbox[4]/2) * size_original[0]
    y2 = (bbox[2] + bbox[4]/2) * size_original[0]

    return [x1, y1, x2, y2]

In [4]:
## 절대좌표 -> YOLO text 파일로 변환
def revert(size, box): # box : [x1,y1,x2,y2]
    dw = 1./size[0] 
    dh = 1./size[1] 
    x = (box[0] + box[2])/2.0 
    y = (box[1] + box[3])/2.0 
    w = box[2] - box[0] 
    h = box[3] - box[1] 
    x = x*dw 
    w = w*dw 
    y = y*dh 
    h = h*dh 
    return [x,y,w,h]

## 1. 90도, 270도(-90도) 회전

In [ ]:
degree = 90
for i in range(2):
    for file in listdir(path_lab+'original\\'):
        print(file) ## 확인용 ##
        boundingbox =  readBox(i, file) ## x_center y_center width height
        class_num = boundingbox[0] ## 생육 단계 (ex. S1, S2, .....)
        
        labelfile = file.replace('.txt', '.jpg')
        image = imageio.imread(path_img + 'original\\' + labelfile)
        
        size = image.shape ## 이미지 사이즈 저장 (y축, x축, BGR) (높이, 넓이, 채널)        
        point = change(boundingbox, size) ## 상대좌표 -> 절대좌표로 변환

        ## 바운딩 박스 좌표
        bbs = BoundingBoxesOnImage([
            BoundingBox(x1 = point[0], y1 = point[1], x2 = point[2], y2 = point[3])
        ], shape = image.shape)

        ## 이미지 변환
        seq = iaa.Sequential([
#             iaa.Multiply((1.2, 1.5)), # 밝기 조절
            iaa.Affine(
                rotate=degree  # 각도 조절 
            )
        ])

        image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

        before = bbs.bounding_boxes[0]
        after = bbs_aug.bounding_boxes[0]

        aug_box = [after.x1, after.y1, after.x2, after.y2]
        aug_box = revert(size, aug_box)
        aug_box = [class_num] + aug_box

        for j in range(len(aug_box)):
            aug_box[j] = str(aug_box[j])
            
        
        with open(f"{path_lab}/augmentation/{file.split('.')[0]}_{i}"+".txt", 'w') as f:
            f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2], ' ', aug_box[3], ' ', aug_box[4],'\n'])
        
        imageio.imwrite(f"{path_img}/augmentation/{file.split('.')[0]}_rotate90" + ".jpg", image_aug)
        
        ## 좌표 변환 출력
        for i in range(len(bbs.bounding_boxes)):
            before = bbs.bounding_boxes[i]
            after = bbs_aug.bounding_boxes[i]
            print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
                i,
                before.x1, before.y1, before.x2, before.y2,
                after.x1, after.y1, after.x2, after.y2)
            )

#         ## 이미지 표시
#         image_before =  bbs.draw_on_image(image, thickness = 20, size = 2)
#         image_after = bbs_aug.draw_on_image(image_aug, size = 2, thickness = 20, color = [0, 0, 255])

#         cv2.imshow('image_before', cv2.resize(image_before, (604, 806)))
#         cv2.imshow('image_after', cv2.resize(image_after, (604, 806)))
        
#         ## 스페이스 바 눌렀을 때 창 닫기
#         while(True):
#             if cv2.waitKey()  == ord(' '):
#                 cv2.destroyAllWindows()
#                 break

0050eb01-S0-20230310-F10-1.txt


C:\Users\USER\AppData\Local\Temp\ipykernel_5976\2393362743.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path_img + 'original\\' + labelfile)


BB 0: (874.0826, 1248.1292, 2019.6194, 2143.5262) -> (1384.4739, 1378.0825, 2279.8708, 2523.6194)
007f46cf-S0-20230317-F14-1.txt
BB 0: (407.1813, 1497.7550, 1292.1221, 2203.2194) -> (1324.7805, 911.1813, 2030.2450, 1796.1221)
00e8098e-S0-20230308-F3-3.txt
BB 0: (1161.8241, 1443.4886, 2052.1939, 2490.8318) -> (1037.1682, 1665.8241, 2084.5115, 2556.1938)
0100772d-S0-20230313-F31-2.txt
BB 0: (548.3375, 1438.0619, 1520.1436, 2197.7927) -> (1330.2073, 1052.3375, 2089.9380, 2024.1437)
01937d0f-S0-20230402-F1-1.txt
BB 0: (602.2679, 1443.4886, 1736.2679, 2420.2853) -> (729.7146, 1484.2679, 1706.5115, 2618.2681)
01d29481-S0-20230411-F11-1.txt
BB 0: (884.9408, 2404.0054, 1579.8636, 2935.8170) -> (592.1831, 1388.9407, 1123.9946, 2083.8635)
020397c9-S0-20230309-F24-2.txt
BB 0: (656.9192, 1579.1548, 1704.7325, 2382.2988) -> (1145.7012, 1160.9192, 1948.8452, 2208.7324)
0228fa6a-S0-20230310-F49-1.txt
BB 0: (1938.1831, 1671.4078, 2839.4111, 2366.0188) -> (1161.9812, 2442.1831, 1856.5922, 3343.4111)
02

BB 0: (271.4542, 1486.9017, 1422.4201, 2214.0727) -> (1313.9272, 775.4542, 2041.0983, 1926.4202)
15da2e43-S0-20230313-F19-3.txt
BB 0: (1579.8636, 2284.6191, 2388.7971, 3272.2692) -> (255.7307, 2083.8635, 1243.3809, 2892.7971)
165bcd2f-S0-20230314-F2-2.txt
BB 0: (966.3770, 993.0767, 1688.4452, 1796.2207) -> (1731.7793, 1470.3770, 2534.9233, 2192.4453)
168fbe3f-S0-20230310-F48-1.txt
BB 0: (1525.5727, 846.5572, 2709.1131, 1807.0740) -> (1720.9260, 2029.5728, 2681.4429, 3213.1130)
16c68ae0-S0-20230315-F3-3.txt
BB 0: (1004.3806, 1448.9152, 1976.1867, 2290.0458) -> (1237.9543, 1508.3806, 2079.0850, 2480.1868)
1779af77-S0-20230313-F36-2.txt
BB 0: (971.8061, 1443.4886, 1742.7361, 2034.9933) -> (1493.0067, 1475.8062, 2084.5115, 2246.7361)
18afb22f-S0-20230308-F29-2.txt
BB 0: (705.7810, 1817.9273, 1569.0054, 2642.7779) -> (885.2222, 1209.7810, 1710.0726, 2073.0054)
18c949b5-S0-20230314-F36-1.txt
BB 0: (939.2316, 1720.2476, 1759.0233, 2409.4320) -> (1118.5679, 1443.2316, 1807.7523, 2263.0234)
18d

BB 0: (1525.5727, 1215.5693, 2215.0664, 2105.5397) -> (1422.4602, 2029.5728, 2312.4307, 2719.0664)
2a1d6b24-S0-20230319-F37-2.txt
BB 0: (1183.9677, 1286.1157, 1741.3683, 1684.9744) -> (1843.0255, 679.9677, 2241.8843, 1237.3683)
2a35d65f-S0-20230313-F14-3.txt
BB 0: (1731.8779, 1921.0336, 2524.5242, 2908.6837) -> (619.3164, 2235.8779, 1606.9663, 3028.5242)
2a7b5603-S0-20230316-F12-1.txt
BB 0: (1004.3806, 2148.9529, 2035.9066, 2805.5774) -> (722.4226, 1508.3806, 1379.0471, 2539.9067)
2a926fb7-S0-20230308-F14-2.txt
BB 0: (1482.1400, 2127.2463, 2557.0987, 3060.6299) -> (467.3701, 1986.1400, 1400.7537, 3061.0986)
2aa8bf91-S0-20230309-F5-1.txt
BB 0: (1351.8420, 1438.0619, 2334.5063, 2197.7927) -> (1330.2073, 1855.8420, 2089.9380, 2838.5063)
2b265c0a-S0-20230315-F12-2.txt
BB 0: (1194.3986, 1839.6339, 2144.4883, 2555.9515) -> (972.0483, 1698.3986, 1688.3661, 2648.4883)
2b69254b-S0-20230315-F13-3.txt
BB 0: (1026.0969, 1720.2476, 1932.7540, 2805.5774) -> (722.4226, 1530.0969, 1807.7523, 2436.7539

...

BB 0: (810.0000, 1632.0000, 1554.0000, 2436.0000) -> (1092.0000, 1314.0000, 1896.0000, 2058.0000)
81a4211d-S0-20230310-F37-3.txt
BB 0: (1004.3806, 2208.6460, 1715.5907, 2995.5101) -> (532.4900, 1508.3806, 1319.3540, 2219.5908)
81af27e6-S0-20230321-F11-3.txt
BB 0: (1367.0555, 1033.7766, 1891.9072, 1729.7443) -> (1798.2557, 863.0555, 2494.2234, 1387.9071)
81ef0a13-S0-20230315-F22-1.txt
BB 0: (504.9048, 1839.6339, 1330.1257, 2452.8452) -> (1075.1548, 1008.9048, 1688.3661, 1834.1257)
827187a7-S0-20230310-F36-2.txt
BB 0: (1102.1041, 1465.1952, 2030.4776, 2159.8062) -> (1368.1938, 1606.1041, 2062.8047, 2534.4775)
8280b8f6-S0-20230316-F37-2.txt
BB 0: (846.9372, 1600.8614, 1905.6086, 2393.1521) -> (1134.8479, 1350.9371, 1927.1387, 2409.6086)
8299ca65-S0-20230308-F58-1.txt
BB 0: (1471.2819, 1850.4872, 2323.6481, 2517.9650) -> (1010.0349, 1975.2819, 1677.5128, 2827.6482)
834bdbe3-S0-20230310-F6-2.txt
BB 0: (1655.8707, 1351.2355, 2752.5458, 2154.3795) -> (1373.6204, 2159.8706, 2176.7646, 3256.545

BB 0: (1129.2496, 1117.8896, 1829.6014, 2007.8600) -> (1520.1400, 1633.2495, 2410.1104, 2333.6016)
92ff80ff-S0-20230308-F28-4.txt
BB 0: (271.4542, 2024.1400, 1161.8241, 2740.4576) -> (787.5425, 775.4542, 1503.8600, 1665.8241)
935bab55-S0-20230307-F4-8.txt
BB 0: (770.9300, 1313.2490, 1856.7469, 2268.3392) -> (1259.6609, 1274.9299, 2214.7510, 2360.7468)
93c0323f-S0-20230321-F37-3.txt
BB 0: (1985.4854, 1220.9960, 2652.7386, 1786.7241) -> (1741.2759, 1481.4854, 2307.0039, 2148.7388)
93df88c8-S0-20230310-F34-3.txt
BB 0: (1655.8707, 1069.0498, 2339.9354, 1931.8869) -> (1596.1130, 2159.8706, 2458.9502, 2843.9353)
94086fff-S0-20230308-F1-5.txt
BB 0: (1037.4975, 1554.7349, 2188.9162, 2315.8223) -> (1212.1777, 533.4974, 1973.2651, 1684.9163)
948cc9af-S0-20230318-F37-3.txt
BB 0: (240.0484, 508.7483, 2449.3078, 2352.4522) -> (1175.5479, -263.9516, 3019.2517, 1945.3079)
9505d19a-S0-20230325-F37-1.txt
BB 0: (884.4115, 1617.1413, 1600.6220, 2474.5518) -> (675.4482, 1766.4115, 1532.8586, 2482.6221)
95

BB 0: (1009.8097, 1872.1938, 1769.8815, 2539.6716) -> (988.3284, 1513.8097, 1655.8062, 2273.8813)
a6dac787-S0-20230310-F7-1.txt
BB 0: (1335.5548, 1774.5141, 1802.4560, 2436.5653) -> (1091.4348, 1839.5548, 1753.4858, 2306.4561)
a78ed860-S0-20230314-F7-3.txt
BB 0: (1471.2819, 906.2503, 2090.1975, 1687.6878) -> (1840.3123, 1975.2819, 2621.7495, 2594.1975)
a8f73e1d-S0-20230313-F28-2.txt
BB 0: (711.2101, 1362.0888, 1607.0090, 2138.0996) -> (1389.9004, 1215.2101, 2165.9111, 2111.0090)
a928c7fb-S0-20230309-F35-1.txt
BB 0: (494.0467, 1807.0740, 1378.9874, 2447.4186) -> (1080.5815, 998.0467, 1720.9260, 1882.9874)
a93e62e3-S0-20230308-F36-3.txt
BB 0: (597.1993, 1698.5410, 1362.7002, 2268.3392) -> (1259.6609, 1101.1992, 1829.4590, 1866.7002)
a9497dcb-S0-20230308-F1-6.txt
BB 0: (866.6155, 1241.3459, 1957.0050, 1990.2234) -> (1537.7766, 362.6155, 2286.6541, 1453.0050)
a988dcf0-S0-20230310-F46-2.txt
BB 0: (1004.3806, 1372.9421, 1851.3178, 2235.7793) -> (1292.2207, 1508.3806, 2155.0579, 2355.3179)
a9

BB 0: (711.2101, 21.7066, 1384.4165, 748.8775) -> (2779.1226, 1215.2101, 3506.2935, 1888.4165)
bd6ba0ae-S0-20230310-F9-2.txt
BB 0: (884.9408, 1823.3540, 1856.7469, 2550.5249) -> (977.4751, 1388.9407, 1704.6460, 2360.7468)
bd759a18-S0-20230309-F6-1.txt
BB 0: (1949.0413, 2349.7389, 2823.1239, 3098.6164) -> (429.3835, 2453.0413, 1178.2610, 3327.1238)
be6e4ec1-S0-20230307-F11-3.txt
BB 0: (553.7666, 1302.3957, 1666.7289, 2328.0323) -> (1199.9678, 1057.7666, 2225.6042, 2170.7290)
bed85e41-S0-20230317-F22-2.txt
BB 0: (1905.6086, 1248.1292, 2893.7020, 2013.2867) -> (1514.7134, 2409.6086, 2279.8708, 3397.7019)
bee73cc2-S0-20230313-F59-1.txt
BB 0: (857.7953, 2110.9664, 1742.7361, 2805.5774) -> (722.4226, 1361.7954, 1417.0337, 2246.7361)
c00b7664-S0-20230315-F10-3.txt
BB 0: (1058.6715, 1991.5801, 1737.3070, 2843.5639) -> (684.4360, 1562.6715, 1536.4199, 2241.3071)
c0521037-S0-20230310-F11-3.txt
BB 0: (2489.9939, 1070.4065, 3824.5005, 2055.3432) -> (1472.6567, 1985.9939, 2457.5935, 3320.5005)
c08f

BB 0: (1226.9731, 2241.2059, 1954.4704, 3190.8694) -> (337.1306, 1730.9730, 1286.7942, 2458.4702)
d1b78ba0-S0-20230310-F28-2.txt
BB 0: (662.3483, 1459.7685, 1655.8707, 2262.9125) -> (1265.0874, 1166.3483, 2068.2314, 2159.8706)
d2b33b91-S0-20230314-F41-2.txt
BB 0: (499.4758, 1351.2355, 1232.4022, 2127.2463) -> (1400.7537, 1003.4758, 2176.7646, 1736.4021)
d2c4c4fc-S0-20230307-F1-1.txt
BB 0: (1398.0000, 1518.0000, 2052.0000, 2274.0000) -> (1254.0000, 1902.0000, 2010.0000, 2556.0000)
d4007161-S0-20230308-F40-3.txt
BB 0: (852.3662, 1410.9287, 1862.1759, 2100.1131) -> (1427.8870, 1356.3662, 2117.0713, 2366.1758)
d4311b87-S0-20230314-F24-2.txt
BB 0: (776.3591, 1416.3553, 1748.1652, 2235.7793) -> (1292.2207, 1280.3591, 2111.6445, 2252.1650)
d45d3cca-S0-20230317-F11-2.txt
BB 0: (1699.3034, 1123.3163, 2660.2513, 1839.6339) -> (1688.3661, 2203.3035, 2404.6836, 3164.2515)
d4751316-S0-20230316-F10-2.txt
BB 0: (1737.3070, 2322.6057, 2687.3968, 3104.0431) -> (423.9570, 2241.3071, 1205.3943, 3191.3967

In [14]:
## 270도 회전(-90도 회전)
degree = 270

for file in listdir(path_lab+'original\\'):
    print(file)
    boundingbox =  readBox(1, file) ## x_center y_center width height
    class_num = boundingbox[0]

    labelfile = file.replace('.txt', '.jpg')
    image = imageio.imread(path_img + 'original\\' + labelfile)

    size = image.shape ## 이미지 사이즈 저장 (y축, x축, BGR) (높이, 넓이, 채널)        
    point = change(boundingbox, size)

    bbs = BoundingBoxesOnImage([
        BoundingBox(x1 = point[0], y1 = point[1], x2 = point[2], y2 = point[3])
    ], shape = image.shape)

    seq = iaa.Sequential([
        iaa.Affine(
            rotate=degree  # 각도 조절 
        )
    ])

    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    before = bbs.bounding_boxes[0]
    after = bbs_aug.bounding_boxes[0]

    aug_box = [after.x1, after.y1, after.x2, after.y2]
    aug_box = revert(size, aug_box)
    aug_box = [class_num] + aug_box

    for j in range(len(aug_box)):
        aug_box[j] = str(aug_box[j])


    with open(f"{path_lab}/augmentation/{file.split('.')[0]}_1"+".txt", 'w') as f:
        f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2], ' ', aug_box[3], ' ', aug_box[4],'\n'])

    imageio.imwrite(f"{path_img}/augmentation/{file.split('.')[0]}_rotateminus90" + ".jpg", image_aug)

    ## 좌표 변환 출력
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            1,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )

#         ## 이미지 표시
#         image_before =  bbs.draw_on_image(image, thickness = 20, size = 2)
#         image_after = bbs_aug.draw_on_image(image_aug, size = 2, thickness = 20, color = [0, 0, 255])

#         cv2.imshow('image_before', cv2.resize(image_before, (604, 806)))
#         cv2.imshow('image_after', cv2.resize(image_after, (604, 806)))

#         while(True):
#             if cv2.waitKey()  == ord(' '):
#                 cv2.destroyAllWindows()
#                 break

0050eb01-S0-20230310-F10-1.txt


C:\Users\USER\AppData\Local\Temp\ipykernel_4008\928409528.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path_img + 'original\\' + labelfile)


BB 1: (874.0826, 1248.1292, 2019.6194, 2143.5262) -> (744.1292, 1508.3806, 1639.5261, 2653.9175)
007f46cf-S0-20230317-F14-1.txt
BB 1: (407.1813, 1497.7550, 1292.1221, 2203.2194) -> (993.7550, 2235.8779, 1699.2195, 3120.8186)
00e8098e-S0-20230308-F3-3.txt
BB 1: (1161.8241, 1443.4886, 2052.1939, 2490.8318) -> (939.4885, 1475.8062, 1986.8318, 2366.1758)
0100772d-S0-20230313-F31-2.txt
BB 1: (548.3375, 1438.0619, 1520.1436, 2197.7927) -> (934.0619, 2007.8563, 1693.7927, 2979.6626)
01937d0f-S0-20230402-F1-1.txt
BB 1: (602.2679, 1443.4886, 1736.2679, 2420.2853) -> (561.4885, 1413.7321, 1538.2854, 2547.7319)
01d29481-S0-20230411-F11-1.txt
BB 1: (884.9408, 2404.0054, 1579.8636, 2935.8170) -> (1900.0054, 1948.1365, 2431.8169, 2643.0593)
020397c9-S0-20230309-F24-2.txt
BB 1: (656.9192, 1579.1548, 1704.7325, 2382.2988) -> (1075.1548, 1823.2675, 1878.2988, 2871.0808)
0228fa6a-S0-20230310-F49-1.txt
BB 1: (1938.1831, 1671.4078, 2839.4111, 2366.0188) -> (1167.4078, 688.5889, 1862.0188, 1589.8169)
029b8

BB 1: (271.4542, 1486.9017, 1422.4201, 2214.0727) -> (982.9017, 2105.5798, 1710.0728, 3256.5459)
15da2e43-S0-20230313-F19-3.txt
BB 1: (1579.8636, 2284.6191, 2388.7971, 3272.2692) -> (1780.6191, 1139.2029, 2768.2693, 1948.1365)
165bcd2f-S0-20230314-F2-2.txt
BB 1: (966.3770, 993.0767, 1688.4452, 1796.2207) -> (489.0767, 1839.5548, 1292.2207, 2561.6230)
168fbe3f-S0-20230310-F48-1.txt
BB 1: (1525.5727, 846.5572, 2709.1131, 1807.0740) -> (342.5572, 818.8870, 1303.0740, 2002.4272)
16c68ae0-S0-20230315-F3-3.txt
BB 1: (1004.3806, 1448.9152, 1976.1867, 2290.0458) -> (944.9152, 1551.8132, 1786.0457, 2523.6194)
1779af77-S0-20230313-F36-2.txt
BB 1: (971.8061, 1443.4886, 1742.7361, 2034.9933) -> (939.4885, 1785.2639, 1530.9933, 2556.1938)
18afb22f-S0-20230308-F29-2.txt
BB 1: (705.7810, 1817.9273, 1569.0054, 2642.7779) -> (1313.9274, 1958.9946, 2138.7778, 2822.2190)
18c949b5-S0-20230314-F36-1.txt
BB 1: (939.2316, 1720.2476, 1759.0233, 2409.4320) -> (1216.2477, 1768.9767, 1905.4321, 2588.7686)
18d285

BB 1: (1525.5727, 1215.5693, 2215.0664, 2105.5397) -> (711.5693, 1312.9336, 1601.5398, 2002.4272)
2a1d6b24-S0-20230319-F37-2.txt
BB 1: (1183.9677, 1286.1157, 1741.3683, 1684.9744) -> (1790.1157, 1786.6317, 2188.9746, 2344.0322)
2a35d65f-S0-20230313-F14-3.txt
BB 1: (1731.8779, 1921.0336, 2524.5242, 2908.6837) -> (1417.0337, 1003.4758, 2404.6836, 1796.1221)
2a7b5603-S0-20230316-F12-1.txt
BB 1: (1004.3806, 2148.9529, 2035.9066, 2805.5774) -> (1644.9529, 1492.0934, 2301.5774, 2523.6194)
2a926fb7-S0-20230308-F14-2.txt
BB 1: (1482.1400, 2127.2463, 2557.0987, 3060.6299) -> (1623.2463, 970.9014, 2556.6299, 2045.8600)
2aa8bf91-S0-20230309-F5-1.txt
BB 1: (1351.8420, 1438.0619, 2334.5063, 2197.7927) -> (934.0619, 1193.4937, 1693.7927, 2176.1580)
2b265c0a-S0-20230315-F12-2.txt
BB 1: (1194.3986, 1839.6339, 2144.4883, 2555.9515) -> (1335.6339, 1383.5117, 2051.9517, 2333.6016)
2b69254b-S0-20230315-F13-3.txt
BB 1: (1026.0969, 1720.2476, 1932.7540, 2805.5774) -> (1216.2477, 1595.2460, 2301.5774, 2501.9

...

BB 1: (1129.2496, 1117.8896, 1829.6014, 2007.8600) -> (613.8896, 1698.3986, 1503.8600, 2398.7505)
92ff80ff-S0-20230308-F28-4.txt
BB 1: (271.4542, 2024.1400, 1161.8241, 2740.4576) -> (1520.1400, 2366.1758, 2236.4575, 3256.5459)
935bab55-S0-20230307-F4-8.txt
BB 1: (770.9300, 1313.2490, 1856.7469, 2268.3392) -> (809.2490, 1671.2532, 1764.3391, 2757.0701)
93c0323f-S0-20230321-F37-3.txt
BB 1: (1985.4854, 1220.9960, 2652.7386, 1786.7241) -> (1724.9960, 875.2612, 2290.7241, 1542.5146)
93df88c8-S0-20230310-F34-3.txt
BB 1: (1655.8707, 1069.0498, 2339.9354, 1931.8869) -> (565.0498, 1188.0647, 1427.8870, 1872.1293)
94086fff-S0-20230308-F1-5.txt
BB 1: (1037.4975, 1554.7349, 2188.9162, 2315.8223) -> (2058.7349, 1339.0837, 2819.8223, 2490.5024)
948cc9af-S0-20230318-F37-3.txt
BB 1: (240.0484, 508.7483, 2449.3078, 2352.4522) -> (1012.7483, 1078.6921, 2856.4521, 3287.9517)
9505d19a-S0-20230325-F37-1.txt
BB 1: (884.4115, 1617.1413, 1600.6220, 2474.5518) -> (735.1414, 1549.3779, 1592.5518, 2265.5884)
951

BB 1: (1009.8097, 1872.1938, 1769.8815, 2539.6716) -> (1368.1938, 1758.1185, 2035.6716, 2518.1904)
a6dac787-S0-20230310-F7-1.txt
BB 1: (1335.5548, 1774.5141, 1802.4560, 2436.5653) -> (1270.5142, 1725.5439, 1932.5652, 2192.4453)
a78ed860-S0-20230314-F7-3.txt
BB 1: (1471.2819, 906.2503, 2090.1975, 1687.6878) -> (402.2504, 1437.8025, 1183.6877, 2056.7183)
a8f73e1d-S0-20230313-F28-2.txt
BB 1: (711.2101, 1362.0888, 1607.0090, 2138.0996) -> (858.0889, 1920.9910, 1634.0996, 2816.7900)
a928c7fb-S0-20230309-F35-1.txt
BB 1: (494.0467, 1807.0740, 1378.9874, 2447.4186) -> (1303.0740, 2149.0127, 1943.4185, 3033.9534)
a93e62e3-S0-20230308-F36-3.txt
BB 1: (597.1993, 1698.5410, 1362.7002, 2268.3392) -> (1194.5410, 2165.2998, 1764.3391, 2930.8008)
a9497dcb-S0-20230308-F1-6.txt
BB 1: (866.6155, 1241.3459, 1957.0050, 1990.2234) -> (1745.3459, 1570.9950, 2494.2234, 2661.3845)
a988dcf0-S0-20230310-F46-2.txt
BB 1: (1004.3806, 1372.9421, 1851.3178, 2235.7793) -> (868.9421, 1676.6823, 1731.7793, 2523.6194)
a9

BB 1: (711.2101, 21.7066, 1384.4165, 748.8775) -> (-482.2934, 2143.5835, 244.8775, 2816.7900)
bd6ba0ae-S0-20230310-F9-2.txt
BB 1: (884.9408, 1823.3540, 1856.7469, 2550.5249) -> (1319.3540, 1671.2532, 2046.5249, 2643.0593)
bd759a18-S0-20230309-F6-1.txt
BB 1: (1949.0413, 2349.7389, 2823.1239, 3098.6164) -> (1845.7390, 704.8762, 2594.6165, 1578.9587)
be6e4ec1-S0-20230307-F11-3.txt
BB 1: (553.7666, 1302.3957, 1666.7289, 2328.0323) -> (798.3958, 1861.2711, 1824.0322, 2974.2334)
bed85e41-S0-20230317-F22-2.txt
BB 1: (1905.6086, 1248.1292, 2893.7020, 2013.2867) -> (744.1292, 634.2981, 1509.2866, 1622.3914)
bee73cc2-S0-20230313-F59-1.txt
BB 1: (857.7953, 2110.9664, 1742.7361, 2805.5774) -> (1606.9663, 1785.2639, 2301.5774, 2670.2046)
c00b7664-S0-20230315-F10-3.txt
BB 1: (1058.6715, 1991.5801, 1737.3070, 2843.5639) -> (1487.5801, 1790.6930, 2339.5640, 2469.3286)
c0521037-S0-20230310-F11-3.txt
BB 1: (2489.9939, 1070.4065, 3824.5005, 2055.3432) -> (1574.4065, -296.5005, 2559.3433, 1038.0061)
c08fe

BB 1: (1226.9731, 2241.2059, 1954.4704, 3190.8694) -> (1737.2058, 1573.5297, 2686.8694, 2301.0269)
d1b78ba0-S0-20230310-F28-2.txt
BB 1: (662.3483, 1459.7685, 1655.8707, 2262.9125) -> (955.7686, 1872.1293, 1758.9126, 2865.6519)
d2b33b91-S0-20230314-F41-2.txt
BB 1: (499.4758, 1351.2355, 1232.4022, 2127.2463) -> (847.2355, 2295.5979, 1623.2463, 3028.5242)
d2c4c4fc-S0-20230307-F1-1.txt
BB 1: (1398.0000, 1518.0000, 2052.0000, 2274.0000) -> (1014.0000, 1476.0000, 1770.0000, 2130.0000)
d4007161-S0-20230308-F40-3.txt
BB 1: (852.3662, 1410.9287, 1862.1759, 2100.1131) -> (906.9287, 1665.8241, 1596.1130, 2675.6338)
d4311b87-S0-20230314-F24-2.txt
BB 1: (776.3591, 1416.3553, 1748.1652, 2235.7793) -> (912.3553, 1779.8348, 1731.7793, 2751.6409)
d45d3cca-S0-20230317-F11-2.txt
BB 1: (1699.3034, 1123.3163, 2660.2513, 1839.6339) -> (619.3163, 867.7485, 1335.6339, 1828.6965)
d4751316-S0-20230316-F10-2.txt
BB 1: (1737.3070, 2322.6057, 2687.3968, 3104.0431) -> (1818.6057, 840.6033, 2600.0430, 1790.6930)
d56

BB 1: (1281.2639, 1145.0229, 2296.5027, 1779.9408) -> (641.0228, 1231.4973, 1275.9408, 2246.7361)
e6f86c89-S0-20230318-F35-3.txt
BB 1: (748.6256, 545.3782, 3246.7568, 2547.8116) -> (1049.3782, 281.2432, 3051.8115, 2779.3745)
e7b7f6c1-S0-20230307-F3-3.txt
BB 1: (1140.0000, 1698.0000, 1752.0000, 2442.0000) -> (1194.0000, 1776.0000, 1938.0000, 2388.0000)
e7eed599-S0-20230308-F9-1.txt
BB 1: (542.9084, 2176.0861, 1471.2819, 2843.5639) -> (1672.0862, 2056.7183, 2339.5640, 2985.0916)
e843153f-S0-20230308-F11-2.txt
BB 1: (1585.2926, 1150.4495, 2633.1059, 2094.6864) -> (646.4496, 894.8940, 1590.6865, 1942.7074)
e8e73bd6-S0-20230323-F37-1.txt
BB 1: (1615.2412, 1803.0040, 2140.0928, 2421.6420) -> (2307.0039, 1387.9072, 2925.6421, 1912.7588)
ea7d92fe-S0-20230313-F9-2.txt
BB 1: (716.6391, 2138.0996, 1655.8707, 2886.9771) -> (1634.0996, 1872.1293, 2382.9771, 2811.3608)
ea7eb852-S0-20230315-F9-2.txt
BB 1: (754.6427, 1535.7416, 1563.5763, 2203.2194) -> (1031.7416, 1964.4237, 1699.2195, 2773.3574)
ea8c

## 2. 좌우반전

In [13]:
for file in listdir(path_lab+'original\\'):
    print(file)
    boundingbox =  readBox(2, file) ## x_center y_center width height
    class_num = boundingbox[0]

    labelfile = file.replace('.txt', '.jpg')
    image = imageio.imread(path_img + 'original\\' + labelfile)

    size = image.shape ## 이미지 사이즈 저장 (y축, x축, BGR) (높이, 넓이, 채널)        
    point = change(boundingbox, size)

    bbs = BoundingBoxesOnImage([
        BoundingBox(x1 = point[0], y1 = point[1], x2 = point[2], y2 = point[3])
    ], shape = image.shape)

    seq = iaa.Sequential([
#             iaa.Multiply((1.2, 1.5)), # 밝기 조절
#         iaa.Affine(
#             rotate=degree  # 각도 조절 
#         )
        iaa.Fliplr(1)    ## 좌우반전
    ])

    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    before = bbs.bounding_boxes[0]
    after = bbs_aug.bounding_boxes[0]

    aug_box = [after.x1, after.y1, after.x2, after.y2]
    aug_box = revert(size, aug_box)
    aug_box = [class_num] + aug_box

    for j in range(len(aug_box)):
        aug_box[j] = str(aug_box[j])


    with open(f"{path_lab}/augmentation/{file.split('.')[0]}_2"+".txt", 'w') as f:
        f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2], ' ', aug_box[3], ' ', aug_box[4],'\n'])

    imageio.imwrite(f"{path_img}/augmentation/{file.split('.')[0]}_flip" + ".jpg", image_aug)

    ## 좌표 변환 출력
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            i,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )

#     ## 이미지 표시
#     image_before =  bbs.draw_on_image(image, thickness = 20, size = 2)
#     image_after = bbs_aug.draw_on_image(image_aug, size = 2, thickness = 20, color = [0, 0, 255])

#     cv2.imshow('image_before', cv2.resize(image_before, (604, 806)))
#     cv2.imshow('image_after', cv2.resize(image_after, (604, 806)))

#     while(True):
#         if cv2.waitKey()  == ord(' '):
#             cv2.destroyAllWindows()
#             break

0050eb01-S0-20230310-F10-1.txt


C:\Users\USER\AppData\Local\Temp\ipykernel_4008\2855577467.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path_img + 'original\\' + labelfile)


BB 0: (874.0826, 1248.1292, 2019.6194, 2143.5262) -> (1004.3806, 1248.1292, 2149.9174, 2143.5262)
007f46cf-S0-20230317-F14-1.txt
BB 0: (407.1813, 1497.7550, 1292.1221, 2203.2194) -> (1731.8779, 1497.7550, 2616.8187, 2203.2194)
00e8098e-S0-20230308-F3-3.txt
BB 0: (1161.8241, 1443.4886, 2052.1939, 2490.8318) -> (971.8061, 1443.4886, 1862.1759, 2490.8318)
0100772d-S0-20230313-F31-2.txt
BB 0: (548.3375, 1438.0619, 1520.1436, 2197.7927) -> (1503.8564, 1438.0619, 2475.6625, 2197.7927)
01937d0f-S0-20230402-F1-1.txt
BB 0: (602.2679, 1443.4886, 1736.2679, 2420.2853) -> (531.7321, 1443.4886, 1665.7321, 2420.2853)
01d29481-S0-20230411-F11-1.txt
BB 0: (884.9408, 2404.0054, 1579.8636, 2935.8170) -> (1444.1364, 2404.0054, 2139.0592, 2935.8170)
020397c9-S0-20230309-F24-2.txt
BB 0: (656.9192, 1579.1548, 1704.7325, 2382.2988) -> (1319.2675, 1579.1548, 2367.0808, 2382.2988)
0228fa6a-S0-20230310-F49-1.txt
BB 0: (1938.1831, 1671.4078, 2839.4111, 2366.0188) -> (184.5889, 1671.4078, 1085.8169, 2366.0188)
02

BB 0: (271.4542, 1486.9017, 1422.4201, 2214.0727) -> (1601.5799, 1486.9017, 2752.5458, 2214.0727)
15da2e43-S0-20230313-F19-3.txt
BB 0: (1579.8636, 2284.6191, 2388.7971, 3272.2692) -> (635.2029, 2284.6191, 1444.1364, 3272.2692)
165bcd2f-S0-20230314-F2-2.txt
BB 0: (966.3770, 993.0767, 1688.4452, 1796.2207) -> (1335.5548, 993.0767, 2057.6230, 1796.2207)
168fbe3f-S0-20230310-F48-1.txt
BB 0: (1525.5727, 846.5572, 2709.1131, 1807.0740) -> (314.8869, 846.5572, 1498.4273, 1807.0740)
16c68ae0-S0-20230315-F3-3.txt
BB 0: (1004.3806, 1448.9152, 1976.1867, 2290.0458) -> (1047.8133, 1448.9152, 2019.6194, 2290.0458)
1779af77-S0-20230313-F36-2.txt
BB 0: (971.8061, 1443.4886, 1742.7361, 2034.9933) -> (1281.2639, 1443.4886, 2052.1939, 2034.9933)
18afb22f-S0-20230308-F29-2.txt
BB 0: (705.7810, 1817.9273, 1569.0054, 2642.7779) -> (1454.9946, 1817.9273, 2318.2190, 2642.7779)
18c949b5-S0-20230314-F36-1.txt
BB 0: (939.2316, 1720.2476, 1759.0233, 2409.4320) -> (1264.9767, 1720.2476, 2084.7684, 2409.4320)
18d2

BB 0: (1525.5727, 1215.5693, 2215.0664, 2105.5397) -> (808.9336, 1215.5693, 1498.4273, 2105.5397)
2a1d6b24-S0-20230319-F37-2.txt
BB 0: (1183.9677, 1286.1157, 1741.3683, 1684.9744) -> (2290.6317, 1286.1157, 2848.0323, 1684.9744)
2a35d65f-S0-20230313-F14-3.txt
BB 0: (1731.8779, 1921.0336, 2524.5242, 2908.6837) -> (499.4758, 1921.0336, 1292.1221, 2908.6837)
2a7b5603-S0-20230316-F12-1.txt
BB 0: (1004.3806, 2148.9529, 2035.9066, 2805.5774) -> (988.0934, 2148.9529, 2019.6194, 2805.5774)
2a926fb7-S0-20230308-F14-2.txt
BB 0: (1482.1400, 2127.2463, 2557.0987, 3060.6299) -> (466.9013, 2127.2463, 1541.8600, 3060.6299)
2aa8bf91-S0-20230309-F5-1.txt
BB 0: (1351.8420, 1438.0619, 2334.5063, 2197.7927) -> (689.4937, 1438.0619, 1672.1580, 2197.7927)
2b265c0a-S0-20230315-F12-2.txt
BB 0: (1194.3986, 1839.6339, 2144.4883, 2555.9515) -> (879.5117, 1839.6339, 1829.6014, 2555.9515)
2b69254b-S0-20230315-F13-3.txt
BB 0: (1026.0969, 1720.2476, 1932.7540, 2805.5774) -> (1091.2460, 1720.2476, 1997.9031, 2805.5774

...

BB 0: (1129.2496, 1117.8896, 1829.6014, 2007.8600) -> (1194.3986, 1117.8896, 1894.7504, 2007.8600)
92ff80ff-S0-20230308-F28-4.txt
BB 0: (271.4542, 2024.1400, 1161.8241, 2740.4576) -> (1862.1759, 2024.1400, 2752.5458, 2740.4576)
935bab55-S0-20230307-F4-8.txt
BB 0: (770.9300, 1313.2490, 1856.7469, 2268.3392) -> (1167.2531, 1313.2490, 2253.0700, 2268.3392)
93c0323f-S0-20230321-F37-3.txt
BB 0: (1985.4854, 1220.9960, 2652.7386, 1786.7241) -> (1379.2614, 1220.9960, 2046.5146, 1786.7241)
93df88c8-S0-20230310-F34-3.txt
BB 0: (1655.8707, 1069.0498, 2339.9354, 1931.8869) -> (684.0646, 1069.0498, 1368.1293, 1931.8869)
94086fff-S0-20230308-F1-5.txt
BB 0: (1037.4975, 1554.7349, 2188.9162, 2315.8223) -> (1843.0838, 1554.7349, 2994.5025, 2315.8223)
948cc9af-S0-20230318-F37-3.txt
BB 0: (240.0484, 508.7483, 2449.3078, 2352.4522) -> (1582.6922, 508.7483, 3791.9516, 2352.4522)
9505d19a-S0-20230325-F37-1.txt
BB 0: (884.4115, 1617.1413, 1600.6220, 2474.5518) -> (667.3780, 1617.1413, 1383.5885, 2474.5518)
9

BB 0: (1009.8097, 1872.1938, 1769.8815, 2539.6716) -> (1254.1185, 1872.1938, 2014.1903, 2539.6716)
a6dac787-S0-20230310-F7-1.txt
BB 0: (1335.5548, 1774.5141, 1802.4560, 2436.5653) -> (1221.5440, 1774.5141, 1688.4452, 2436.5653)
a78ed860-S0-20230314-F7-3.txt
BB 0: (1471.2819, 906.2503, 2090.1975, 1687.6878) -> (933.8025, 906.2503, 1552.7181, 1687.6878)
a8f73e1d-S0-20230313-F28-2.txt
BB 0: (711.2101, 1362.0888, 1607.0090, 2138.0996) -> (1416.9910, 1362.0888, 2312.7899, 2138.0996)
a928c7fb-S0-20230309-F35-1.txt
BB 0: (494.0467, 1807.0740, 1378.9874, 2447.4186) -> (1645.0126, 1807.0740, 2529.9533, 2447.4186)
a93e62e3-S0-20230308-F36-3.txt
BB 0: (597.1993, 1698.5410, 1362.7002, 2268.3392) -> (1661.2998, 1698.5410, 2426.8007, 2268.3392)
a9497dcb-S0-20230308-F1-6.txt
BB 0: (866.6155, 1241.3459, 1957.0050, 1990.2234) -> (2074.9950, 1241.3459, 3165.3845, 1990.2234)
a988dcf0-S0-20230310-F46-2.txt
BB 0: (1004.3806, 1372.9421, 1851.3178, 2235.7793) -> (1172.6822, 1372.9421, 2019.6194, 2235.7793)
a

BB 0: (711.2101, 21.7066, 1384.4165, 748.8775) -> (1639.5835, 21.7066, 2312.7899, 748.8775)
bd6ba0ae-S0-20230310-F9-2.txt
BB 0: (884.9408, 1823.3540, 1856.7469, 2550.5249) -> (1167.2531, 1823.3540, 2139.0592, 2550.5249)
bd759a18-S0-20230309-F6-1.txt
BB 0: (1949.0413, 2349.7389, 2823.1239, 3098.6164) -> (200.8761, 2349.7389, 1074.9587, 3098.6164)
be6e4ec1-S0-20230307-F11-3.txt
BB 0: (553.7666, 1302.3957, 1666.7289, 2328.0323) -> (1357.2711, 1302.3957, 2470.2334, 2328.0323)
bed85e41-S0-20230317-F22-2.txt
BB 0: (1905.6086, 1248.1292, 2893.7020, 2013.2867) -> (130.2980, 1248.1292, 1118.3914, 2013.2867)
bee73cc2-S0-20230313-F59-1.txt
BB 0: (857.7953, 2110.9664, 1742.7361, 2805.5774) -> (1281.2639, 2110.9664, 2166.2047, 2805.5774)
c00b7664-S0-20230315-F10-3.txt
BB 0: (1058.6715, 1991.5801, 1737.3070, 2843.5639) -> (1286.6930, 1991.5801, 1965.3285, 2843.5639)
c0521037-S0-20230310-F11-3.txt
BB 0: (2489.9939, 1070.4065, 3824.5005, 2055.3432) -> (207.4995, 1070.4065, 1542.0061, 2055.3432)
c08fe7

BB 0: (1226.9731, 2241.2059, 1954.4704, 3190.8694) -> (1069.5296, 2241.2059, 1797.0269, 3190.8694)
d1b78ba0-S0-20230310-F28-2.txt
BB 0: (662.3483, 1459.7685, 1655.8707, 2262.9125) -> (1368.1293, 1459.7685, 2361.6517, 2262.9125)
d2b33b91-S0-20230314-F41-2.txt
BB 0: (499.4758, 1351.2355, 1232.4022, 2127.2463) -> (1791.5978, 1351.2355, 2524.5242, 2127.2463)
d2c4c4fc-S0-20230307-F1-1.txt
BB 0: (1398.0000, 1518.0000, 2052.0000, 2274.0000) -> (972.0000, 1518.0000, 1626.0000, 2274.0000)
d4007161-S0-20230308-F40-3.txt
BB 0: (852.3662, 1410.9287, 1862.1759, 2100.1131) -> (1161.8241, 1410.9287, 2171.6338, 2100.1131)
d4311b87-S0-20230314-F24-2.txt
BB 0: (776.3591, 1416.3553, 1748.1652, 2235.7793) -> (1275.8348, 1416.3553, 2247.6409, 2235.7793)
d45d3cca-S0-20230317-F11-2.txt
BB 0: (1699.3034, 1123.3163, 2660.2513, 1839.6339) -> (363.7487, 1123.3163, 1324.6966, 1839.6339)
d4751316-S0-20230316-F10-2.txt
BB 0: (1737.3070, 2322.6057, 2687.3968, 3104.0431) -> (336.6032, 2322.6057, 1286.6930, 3104.0431)

BB 0: (1281.2639, 1145.0229, 2296.5027, 1779.9408) -> (727.4973, 1145.0229, 1742.7361, 1779.9408)
e6f86c89-S0-20230318-F35-3.txt
BB 0: (748.6256, 545.3782, 3246.7568, 2547.8116) -> (785.2432, 545.3782, 3283.3744, 2547.8116)
e7b7f6c1-S0-20230307-F3-3.txt
BB 0: (1140.0000, 1698.0000, 1752.0000, 2442.0000) -> (1272.0000, 1698.0000, 1884.0000, 2442.0000)
e7eed599-S0-20230308-F9-1.txt
BB 0: (542.9084, 2176.0861, 1471.2819, 2843.5639) -> (1552.7181, 2176.0861, 2481.0916, 2843.5639)
e843153f-S0-20230308-F11-2.txt
BB 0: (1585.2926, 1150.4495, 2633.1059, 2094.6864) -> (390.8941, 1150.4495, 1438.7074, 2094.6864)
e8e73bd6-S0-20230323-F37-1.txt
BB 0: (1615.2412, 1803.0040, 2140.0928, 2421.6420) -> (1891.9072, 1803.0040, 2416.7588, 2421.6420)
ea7d92fe-S0-20230313-F9-2.txt
BB 0: (716.6391, 2138.0996, 1655.8707, 2886.9771) -> (1368.1293, 2138.0996, 2307.3609, 2886.9771)
ea7eb852-S0-20230315-F9-2.txt
BB 0: (754.6427, 1535.7416, 1563.5763, 2203.2194) -> (1460.4237, 1535.7416, 2269.3573, 2203.2194)
ea8c

## 3. 좌우반전, 90도 회전

In [20]:
for file in listdir(path_lab+'original\\'):
    print(file)
    boundingbox =  readBox(3, file) ## x_center y_center width height
    class_num = boundingbox[0]

    labelfile = file.replace('.txt', '.jpg')
    image = imageio.imread(path_img + 'original\\' + labelfile)

    size = image.shape ## 이미지 사이즈 저장 (y축, x축, BGR) (높이, 넓이, 채널)        
    point = change(boundingbox, size)

    bbs = BoundingBoxesOnImage([
        BoundingBox(x1 = point[0], y1 = point[1], x2 = point[2], y2 = point[3])
    ], shape = image.shape)

    seq = iaa.Sequential([
        iaa.Affine(rotate = 90),  # 각도 조절 
        iaa.Fliplr(1)
    ])

    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    before = bbs.bounding_boxes[0]
    after = bbs_aug.bounding_boxes[0]

    aug_box = [after.x1, after.y1, after.x2, after.y2]
    aug_box = revert(size, aug_box)
    aug_box = [class_num] + aug_box

    for j in range(len(aug_box)):
        aug_box[j] = str(aug_box[j])


    with open(f"{path_lab}/augmentation/{file.split('.')[0]}_3"+".txt", 'w') as f:
        f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2], ' ', aug_box[3], ' ', aug_box[4],'\n'])

    imageio.imwrite(f"{path_img}/augmentation/{file.split('.')[0]}_flip_rotate90" + ".jpg", image_aug)

    ## 좌표 변환 출력
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            3,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )

#     ## 이미지 표시
#     image_before =  bbs.draw_on_image(image, thickness = 20, size = 2)
#     image_after = bbs_aug.draw_on_image(image_aug, size = 2, thickness = 20, color = [0, 0, 255])

#     cv2.imshow('image_before', cv2.resize(image_before, (604, 806)))
#     cv2.imshow('image_after', cv2.resize(image_after, (604, 806)))

#     while(True):
#         if cv2.waitKey()  == ord(' '):
#             cv2.destroyAllWindows()
#             break

0050eb01-S0-20230310-F10-1.txt


C:\Users\USER\AppData\Local\Temp\ipykernel_4008\3692400673.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path_img + 'original\\' + labelfile)


BB 3: (874.0826, 1248.1292, 2019.6194, 2143.5262) -> (744.1292, 1378.0825, 1639.5261, 2523.6194)
007f46cf-S0-20230317-F14-1.txt
BB 3: (407.1813, 1497.7550, 1292.1221, 2203.2194) -> (993.7550, 911.1813, 1699.2195, 1796.1221)
00e8098e-S0-20230308-F3-3.txt
BB 3: (1161.8241, 1443.4886, 2052.1939, 2490.8318) -> (939.4885, 1665.8241, 1986.8318, 2556.1938)
0100772d-S0-20230313-F31-2.txt
BB 3: (548.3375, 1438.0619, 1520.1436, 2197.7927) -> (934.0620, 1052.3375, 1693.7927, 2024.1437)
01937d0f-S0-20230402-F1-1.txt
BB 3: (602.2679, 1443.4886, 1736.2679, 2420.2853) -> (561.4885, 1484.2679, 1538.2854, 2618.2681)
01d29481-S0-20230411-F11-1.txt
BB 3: (884.9408, 2404.0054, 1579.8636, 2935.8170) -> (1900.0054, 1388.9407, 2431.8169, 2083.8635)
020397c9-S0-20230309-F24-2.txt
BB 3: (656.9192, 1579.1548, 1704.7325, 2382.2988) -> (1075.1548, 1160.9192, 1878.2988, 2208.7324)
0228fa6a-S0-20230310-F49-1.txt
BB 3: (1938.1831, 1671.4078, 2839.4111, 2366.0188) -> (1167.4078, 2442.1831, 1862.0188, 3343.4111)
029b8

BB 3: (271.4542, 1486.9017, 1422.4201, 2214.0727) -> (982.9017, 775.4542, 1710.0728, 1926.4202)
15da2e43-S0-20230313-F19-3.txt
BB 3: (1579.8636, 2284.6191, 2388.7971, 3272.2692) -> (1780.6191, 2083.8635, 2768.2693, 2892.7971)
165bcd2f-S0-20230314-F2-2.txt
BB 3: (966.3770, 993.0767, 1688.4452, 1796.2207) -> (489.0767, 1470.3770, 1292.2207, 2192.4453)
168fbe3f-S0-20230310-F48-1.txt
BB 3: (1525.5727, 846.5572, 2709.1131, 1807.0740) -> (342.5571, 2029.5728, 1303.0740, 3213.1130)
16c68ae0-S0-20230315-F3-3.txt
BB 3: (1004.3806, 1448.9152, 1976.1867, 2290.0458) -> (944.9150, 1508.3806, 1786.0457, 2480.1868)
1779af77-S0-20230313-F36-2.txt
BB 3: (971.8061, 1443.4886, 1742.7361, 2034.9933) -> (939.4885, 1475.8062, 1530.9933, 2246.7361)
18afb22f-S0-20230308-F29-2.txt
BB 3: (705.7810, 1817.9273, 1569.0054, 2642.7779) -> (1313.9274, 1209.7810, 2138.7778, 2073.0054)
18c949b5-S0-20230314-F36-1.txt
BB 3: (939.2316, 1720.2476, 1759.0233, 2409.4320) -> (1216.2477, 1443.2316, 1905.4321, 2263.0234)
18d285

BB 3: (1525.5727, 1215.5693, 2215.0664, 2105.5397) -> (711.5693, 2029.5728, 1601.5398, 2719.0664)
2a1d6b24-S0-20230319-F37-2.txt
BB 3: (1183.9677, 1286.1157, 1741.3683, 1684.9744) -> (1790.1157, 679.9677, 2188.9745, 1237.3683)
2a35d65f-S0-20230313-F14-3.txt
BB 3: (1731.8779, 1921.0336, 2524.5242, 2908.6837) -> (1417.0337, 2235.8779, 2404.6836, 3028.5242)
2a7b5603-S0-20230316-F12-1.txt
BB 3: (1004.3806, 2148.9529, 2035.9066, 2805.5774) -> (1644.9529, 1508.3806, 2301.5774, 2539.9067)
2a926fb7-S0-20230308-F14-2.txt
BB 3: (1482.1400, 2127.2463, 2557.0987, 3060.6299) -> (1623.2463, 1986.1400, 2556.6299, 3061.0986)
2aa8bf91-S0-20230309-F5-1.txt
BB 3: (1351.8420, 1438.0619, 2334.5063, 2197.7927) -> (934.0620, 1855.8420, 1693.7927, 2838.5063)
2b265c0a-S0-20230315-F12-2.txt
BB 3: (1194.3986, 1839.6339, 2144.4883, 2555.9515) -> (1335.6339, 1698.3986, 2051.9517, 2648.4883)
2b69254b-S0-20230315-F13-3.txt
BB 3: (1026.0969, 1720.2476, 1932.7540, 2805.5774) -> (1216.2477, 1530.0969, 2301.5774, 2436.7

...

BB 3: (1129.2496, 1117.8896, 1829.6014, 2007.8600) -> (613.8896, 1633.2495, 1503.8600, 2333.6016)
92ff80ff-S0-20230308-F28-4.txt
BB 3: (271.4542, 2024.1400, 1161.8241, 2740.4576) -> (1520.1400, 775.4542, 2236.4575, 1665.8241)
935bab55-S0-20230307-F4-8.txt
BB 3: (770.9300, 1313.2490, 1856.7469, 2268.3392) -> (809.2490, 1274.9299, 1764.3391, 2360.7468)
93c0323f-S0-20230321-F37-3.txt
BB 3: (1985.4854, 1220.9960, 2652.7386, 1786.7241) -> (1724.9961, 1481.4854, 2290.7241, 2148.7388)
93df88c8-S0-20230310-F34-3.txt
BB 3: (1655.8707, 1069.0498, 2339.9354, 1931.8869) -> (565.0498, 2159.8706, 1427.8870, 2843.9353)
94086fff-S0-20230308-F1-5.txt
BB 3: (1037.4975, 1554.7349, 2188.9162, 2315.8223) -> (2058.7349, 533.4974, 2819.8223, 1684.9163)
948cc9af-S0-20230318-F37-3.txt
BB 3: (240.0484, 508.7483, 2449.3078, 2352.4522) -> (1012.7483, -263.9516, 2856.4521, 1945.3079)
9505d19a-S0-20230325-F37-1.txt
BB 3: (884.4115, 1617.1413, 1600.6220, 2474.5518) -> (735.1414, 1766.4115, 1592.5518, 2482.6221)
9515

BB 3: (1009.8097, 1872.1938, 1769.8815, 2539.6716) -> (1368.1938, 1513.8097, 2035.6716, 2273.8813)
a6dac787-S0-20230310-F7-1.txt
BB 3: (1335.5548, 1774.5141, 1802.4560, 2436.5653) -> (1270.5142, 1839.5548, 1932.5652, 2306.4561)
a78ed860-S0-20230314-F7-3.txt
BB 3: (1471.2819, 906.2503, 2090.1975, 1687.6878) -> (402.2505, 1975.2819, 1183.6877, 2594.1975)
a8f73e1d-S0-20230313-F28-2.txt
BB 3: (711.2101, 1362.0888, 1607.0090, 2138.0996) -> (858.0889, 1215.2101, 1634.0996, 2111.0090)
a928c7fb-S0-20230309-F35-1.txt
BB 3: (494.0467, 1807.0740, 1378.9874, 2447.4186) -> (1303.0740, 998.0467, 1943.4185, 1882.9874)
a93e62e3-S0-20230308-F36-3.txt
BB 3: (597.1993, 1698.5410, 1362.7002, 2268.3392) -> (1194.5410, 1101.1992, 1764.3391, 1866.7002)
a9497dcb-S0-20230308-F1-6.txt
BB 3: (866.6155, 1241.3459, 1957.0050, 1990.2234) -> (1745.3459, 362.6155, 2494.2234, 1453.0050)
a988dcf0-S0-20230310-F46-2.txt
BB 3: (1004.3806, 1372.9421, 1851.3178, 2235.7793) -> (868.9421, 1508.3806, 1731.7793, 2355.3179)
a9df

BB 3: (711.2101, 21.7066, 1384.4165, 748.8775) -> (-482.2935, 1215.2101, 244.8774, 1888.4165)
bd6ba0ae-S0-20230310-F9-2.txt
BB 3: (884.9408, 1823.3540, 1856.7469, 2550.5249) -> (1319.3540, 1388.9407, 2046.5249, 2360.7468)
bd759a18-S0-20230309-F6-1.txt
BB 3: (1949.0413, 2349.7389, 2823.1239, 3098.6164) -> (1845.7390, 2453.0413, 2594.6165, 3327.1238)
be6e4ec1-S0-20230307-F11-3.txt
BB 3: (553.7666, 1302.3957, 1666.7289, 2328.0323) -> (798.3958, 1057.7666, 1824.0322, 2170.7290)
bed85e41-S0-20230317-F22-2.txt
BB 3: (1905.6086, 1248.1292, 2893.7020, 2013.2867) -> (744.1292, 2409.6086, 1509.2866, 3397.7019)
bee73cc2-S0-20230313-F59-1.txt
BB 3: (857.7953, 2110.9664, 1742.7361, 2805.5774) -> (1606.9663, 1361.7954, 2301.5774, 2246.7361)
c00b7664-S0-20230315-F10-3.txt
BB 3: (1058.6715, 1991.5801, 1737.3070, 2843.5639) -> (1487.5801, 1562.6715, 2339.5640, 2241.3071)
c0521037-S0-20230310-F11-3.txt
BB 3: (2489.9939, 1070.4065, 3824.5005, 2055.3432) -> (1574.4065, 1985.9939, 2559.3433, 3320.5005)
c08

BB 3: (1226.9731, 2241.2059, 1954.4704, 3190.8694) -> (1737.2058, 1730.9730, 2686.8694, 2458.4702)
d1b78ba0-S0-20230310-F28-2.txt
BB 3: (662.3483, 1459.7685, 1655.8707, 2262.9125) -> (955.7686, 1166.3483, 1758.9126, 2159.8706)
d2b33b91-S0-20230314-F41-2.txt
BB 3: (499.4758, 1351.2355, 1232.4022, 2127.2463) -> (847.2354, 1003.4758, 1623.2463, 1736.4021)
d2c4c4fc-S0-20230307-F1-1.txt
BB 3: (1398.0000, 1518.0000, 2052.0000, 2274.0000) -> (1014.0000, 1902.0000, 1770.0000, 2556.0000)
d4007161-S0-20230308-F40-3.txt
BB 3: (852.3662, 1410.9287, 1862.1759, 2100.1131) -> (906.9287, 1356.3662, 1596.1130, 2366.1758)
d4311b87-S0-20230314-F24-2.txt
BB 3: (776.3591, 1416.3553, 1748.1652, 2235.7793) -> (912.3555, 1280.3591, 1731.7793, 2252.1650)
d45d3cca-S0-20230317-F11-2.txt
BB 3: (1699.3034, 1123.3163, 2660.2513, 1839.6339) -> (619.3164, 2203.3035, 1335.6339, 3164.2515)
d4751316-S0-20230316-F10-2.txt
BB 3: (1737.3070, 2322.6057, 2687.3968, 3104.0431) -> (1818.6057, 2241.3071, 2600.0430, 3191.3967)
d

BB 3: (1281.2639, 1145.0229, 2296.5027, 1779.9408) -> (641.0229, 1785.2639, 1275.9408, 2800.5027)
e6f86c89-S0-20230318-F35-3.txt
BB 3: (748.6256, 545.3782, 3246.7568, 2547.8116) -> (1049.3782, 244.6256, 3051.8115, 2742.7568)
e7b7f6c1-S0-20230307-F3-3.txt
BB 3: (1140.0000, 1698.0000, 1752.0000, 2442.0000) -> (1194.0000, 1644.0000, 1938.0000, 2256.0000)
e7eed599-S0-20230308-F9-1.txt
BB 3: (542.9084, 2176.0861, 1471.2819, 2843.5639) -> (1672.0862, 1046.9084, 2339.5640, 1975.2819)
e843153f-S0-20230308-F11-2.txt
BB 3: (1585.2926, 1150.4495, 2633.1059, 2094.6864) -> (646.4497, 2089.2925, 1590.6865, 3137.1060)
e8e73bd6-S0-20230323-F37-1.txt
BB 3: (1615.2412, 1803.0040, 2140.0928, 2421.6420) -> (2307.0040, 1111.2412, 2925.6421, 1636.0928)
ea7d92fe-S0-20230313-F9-2.txt
BB 3: (716.6391, 2138.0996, 1655.8707, 2886.9771) -> (1634.0996, 1220.6392, 2382.9771, 2159.8706)
ea7eb852-S0-20230315-F9-2.txt
BB 3: (754.6427, 1535.7416, 1563.5763, 2203.2194) -> (1031.7416, 1258.6427, 1699.2195, 2067.5762)
ea8

## 4. 좌우반전, 270도(-90도) 회전

In [21]:
for file in listdir(path_lab+'original\\'):
    print(file)
    boundingbox =  readBox(4, file) ## x_center y_center width height
    class_num = boundingbox[0]

    labelfile = file.replace('.txt', '.jpg')
    image = imageio.imread(path_img + 'original\\' + labelfile)

    size = image.shape ## 이미지 사이즈 저장 (y축, x축, BGR) (높이, 넓이, 채널)        
    point = change(boundingbox, size)

    bbs = BoundingBoxesOnImage([
        BoundingBox(x1 = point[0], y1 = point[1], x2 = point[2], y2 = point[3])
    ], shape = image.shape)

    seq = iaa.Sequential([
        iaa.Affine(rotate = 270),  # 각도 조절 
        iaa.Fliplr(1)
    ])

    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    before = bbs.bounding_boxes[0]
    after = bbs_aug.bounding_boxes[0]

    aug_box = [after.x1, after.y1, after.x2, after.y2]
    aug_box = revert(size, aug_box)
    aug_box = [class_num] + aug_box

    for j in range(len(aug_box)):
        aug_box[j] = str(aug_box[j])


    ## 파일 저장
    ## 라벨링 텍스트 파일 저장(Yolo 버전)
    with open(f"{path_lab}/augmentation/{file.split('.')[0]}_4"+".txt", 'w') as f:
        f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2], ' ', aug_box[3], ' ', aug_box[4],'\n'])

    ## 증강된 이미지 파일 저장
    imageio.imwrite(f"{path_img}/augmentation/{file.split('.')[0]}_flip_rotateminus90" + ".jpg", image_aug)

    ## 좌표 변환 출력
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            3,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )

#     ## 이미지 표시
#     image_before =  bbs.draw_on_image(image, thickness = 20, size = 2)
#     image_after = bbs_aug.draw_on_image(image_aug, size = 2, thickness = 20, color = [0, 0, 255])

#     cv2.imshow('image_before', cv2.resize(image_before, (604, 806)))
#     cv2.imshow('image_after', cv2.resize(image_after, (604, 806)))

#     while(True):
#         if cv2.waitKey()  == ord(' '):
#             cv2.destroyAllWindows()
#             break

0050eb01-S0-20230310-F10-1.txt


C:\Users\USER\AppData\Local\Temp\ipykernel_4008\3362019706.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path_img + 'original\\' + labelfile)


BB 3: (874.0826, 1248.1292, 2019.6194, 2143.5262) -> (1384.4739, 1508.3806, 2279.8708, 2653.9175)
007f46cf-S0-20230317-F14-1.txt
BB 3: (407.1813, 1497.7550, 1292.1221, 2203.2194) -> (1324.7805, 2235.8779, 2030.2450, 3120.8186)
00e8098e-S0-20230308-F3-3.txt
BB 3: (1161.8241, 1443.4886, 2052.1939, 2490.8318) -> (1037.1682, 1475.8062, 2084.5115, 2366.1758)
0100772d-S0-20230313-F31-2.txt
BB 3: (548.3375, 1438.0619, 1520.1436, 2197.7927) -> (1330.2073, 2007.8563, 2089.9381, 2979.6626)
01937d0f-S0-20230402-F1-1.txt
BB 3: (602.2679, 1443.4886, 1736.2679, 2420.2853) -> (729.7146, 1413.7321, 1706.5115, 2547.7319)
01d29481-S0-20230411-F11-1.txt
BB 3: (884.9408, 2404.0054, 1579.8636, 2935.8170) -> (592.1831, 1948.1365, 1123.9946, 2643.0593)
020397c9-S0-20230309-F24-2.txt
BB 3: (656.9192, 1579.1548, 1704.7325, 2382.2988) -> (1145.7012, 1823.2675, 1948.8452, 2871.0808)
0228fa6a-S0-20230310-F49-1.txt
BB 3: (1938.1831, 1671.4078, 2839.4111, 2366.0188) -> (1161.9812, 688.5889, 1856.5922, 1589.8169)
02

BB 3: (271.4542, 1486.9017, 1422.4201, 2214.0727) -> (1313.9272, 2105.5798, 2041.0983, 3256.5459)
15da2e43-S0-20230313-F19-3.txt
BB 3: (1579.8636, 2284.6191, 2388.7971, 3272.2692) -> (255.7307, 1139.2029, 1243.3809, 1948.1365)
165bcd2f-S0-20230314-F2-2.txt
BB 3: (966.3770, 993.0767, 1688.4452, 1796.2207) -> (1731.7793, 1839.5548, 2534.9233, 2561.6230)
168fbe3f-S0-20230310-F48-1.txt
BB 3: (1525.5727, 846.5572, 2709.1131, 1807.0740) -> (1720.9260, 818.8870, 2681.4428, 2002.4272)
16c68ae0-S0-20230315-F3-3.txt
BB 3: (1004.3806, 1448.9152, 1976.1867, 2290.0458) -> (1237.9543, 1551.8132, 2079.0848, 2523.6194)
1779af77-S0-20230313-F36-2.txt
BB 3: (971.8061, 1443.4886, 1742.7361, 2034.9933) -> (1493.0067, 1785.2639, 2084.5115, 2556.1938)
18afb22f-S0-20230308-F29-2.txt
BB 3: (705.7810, 1817.9273, 1569.0054, 2642.7779) -> (885.2222, 1958.9946, 1710.0726, 2822.2190)
18c949b5-S0-20230314-F36-1.txt
BB 3: (939.2316, 1720.2476, 1759.0233, 2409.4320) -> (1118.5679, 1768.9767, 1807.7523, 2588.7686)
18d

BB 3: (1525.5727, 1215.5693, 2215.0664, 2105.5397) -> (1422.4602, 1312.9336, 2312.4307, 2002.4272)
2a1d6b24-S0-20230319-F37-2.txt
BB 3: (1183.9677, 1286.1157, 1741.3683, 1684.9744) -> (1843.0254, 1786.6317, 2241.8843, 2344.0322)
2a35d65f-S0-20230313-F14-3.txt
BB 3: (1731.8779, 1921.0336, 2524.5242, 2908.6837) -> (619.3164, 1003.4758, 1606.9663, 1796.1221)
2a7b5603-S0-20230316-F12-1.txt
BB 3: (1004.3806, 2148.9529, 2035.9066, 2805.5774) -> (722.4226, 1492.0934, 1379.0471, 2523.6194)
2a926fb7-S0-20230308-F14-2.txt
BB 3: (1482.1400, 2127.2463, 2557.0987, 3060.6299) -> (467.3701, 970.9014, 1400.7537, 2045.8600)
2aa8bf91-S0-20230309-F5-1.txt
BB 3: (1351.8420, 1438.0619, 2334.5063, 2197.7927) -> (1330.2073, 1193.4937, 2089.9381, 2176.1580)
2b265c0a-S0-20230315-F12-2.txt
BB 3: (1194.3986, 1839.6339, 2144.4883, 2555.9515) -> (972.0483, 1383.5117, 1688.3661, 2333.6016)
2b69254b-S0-20230315-F13-3.txt
BB 3: (1026.0969, 1720.2476, 1932.7540, 2805.5774) -> (722.4226, 1595.2460, 1807.7523, 2501.9031

...

BB 3: (1129.2496, 1117.8896, 1829.6014, 2007.8600) -> (1520.1400, 1698.3986, 2410.1104, 2398.7505)
92ff80ff-S0-20230308-F28-4.txt
BB 3: (271.4542, 2024.1400, 1161.8241, 2740.4576) -> (787.5425, 2366.1758, 1503.8600, 3256.5459)
935bab55-S0-20230307-F4-8.txt
BB 3: (770.9300, 1313.2490, 1856.7469, 2268.3392) -> (1259.6609, 1671.2532, 2214.7510, 2757.0701)
93c0323f-S0-20230321-F37-3.txt
BB 3: (1985.4854, 1220.9960, 2652.7386, 1786.7241) -> (1741.2759, 875.2612, 2307.0040, 1542.5146)
93df88c8-S0-20230310-F34-3.txt
BB 3: (1655.8707, 1069.0498, 2339.9354, 1931.8869) -> (1596.1130, 1188.0647, 2458.9502, 1872.1293)
94086fff-S0-20230308-F1-5.txt
BB 3: (1037.4975, 1554.7349, 2188.9162, 2315.8223) -> (1212.1777, 1339.0837, 1973.2651, 2490.5024)
948cc9af-S0-20230318-F37-3.txt
BB 3: (240.0484, 508.7483, 2449.3078, 2352.4522) -> (1175.5479, 1078.6921, 3019.2517, 3287.9517)
9505d19a-S0-20230325-F37-1.txt
BB 3: (884.4115, 1617.1413, 1600.6220, 2474.5518) -> (675.4482, 1549.3779, 1532.8586, 2265.5884)
9

BB 3: (1009.8097, 1872.1938, 1769.8815, 2539.6716) -> (988.3284, 1758.1185, 1655.8062, 2518.1904)
a6dac787-S0-20230310-F7-1.txt
BB 3: (1335.5548, 1774.5141, 1802.4560, 2436.5653) -> (1091.4348, 1725.5439, 1753.4858, 2192.4453)
a78ed860-S0-20230314-F7-3.txt
BB 3: (1471.2819, 906.2503, 2090.1975, 1687.6878) -> (1840.3123, 1437.8025, 2621.7496, 2056.7183)
a8f73e1d-S0-20230313-F28-2.txt
BB 3: (711.2101, 1362.0888, 1607.0090, 2138.0996) -> (1389.9004, 1920.9910, 2165.9111, 2816.7900)
a928c7fb-S0-20230309-F35-1.txt
BB 3: (494.0467, 1807.0740, 1378.9874, 2447.4186) -> (1080.5815, 2149.0127, 1720.9260, 3033.9534)
a93e62e3-S0-20230308-F36-3.txt
BB 3: (597.1993, 1698.5410, 1362.7002, 2268.3392) -> (1259.6609, 2165.2998, 1829.4590, 2930.8008)
a9497dcb-S0-20230308-F1-6.txt
BB 3: (866.6155, 1241.3459, 1957.0050, 1990.2234) -> (1537.7766, 1570.9950, 2286.6541, 2661.3845)
a988dcf0-S0-20230310-F46-2.txt
BB 3: (1004.3806, 1372.9421, 1851.3178, 2235.7793) -> (1292.2207, 1676.6823, 2155.0579, 2523.6194)


BB 3: (711.2101, 21.7066, 1384.4165, 748.8775) -> (2779.1225, 2143.5835, 3506.2934, 2816.7900)
bd6ba0ae-S0-20230310-F9-2.txt
BB 3: (884.9408, 1823.3540, 1856.7469, 2550.5249) -> (977.4751, 1671.2532, 1704.6460, 2643.0593)
bd759a18-S0-20230309-F6-1.txt
BB 3: (1949.0413, 2349.7389, 2823.1239, 3098.6164) -> (429.3835, 704.8762, 1178.2610, 1578.9587)
be6e4ec1-S0-20230307-F11-3.txt
BB 3: (553.7666, 1302.3957, 1666.7289, 2328.0323) -> (1199.9678, 1861.2711, 2225.6042, 2974.2334)
bed85e41-S0-20230317-F22-2.txt
BB 3: (1905.6086, 1248.1292, 2893.7020, 2013.2867) -> (1514.7134, 634.2981, 2279.8708, 1622.3914)
bee73cc2-S0-20230313-F59-1.txt
BB 3: (857.7953, 2110.9664, 1742.7361, 2805.5774) -> (722.4226, 1785.2639, 1417.0337, 2670.2046)
c00b7664-S0-20230315-F10-3.txt
BB 3: (1058.6715, 1991.5801, 1737.3070, 2843.5639) -> (684.4360, 1790.6930, 1536.4199, 2469.3286)
c0521037-S0-20230310-F11-3.txt
BB 3: (2489.9939, 1070.4065, 3824.5005, 2055.3432) -> (1472.6567, -296.5005, 2457.5935, 1038.0061)
c08fe7

BB 3: (1226.9731, 2241.2059, 1954.4704, 3190.8694) -> (337.1306, 1573.5297, 1286.7942, 2301.0269)
d1b78ba0-S0-20230310-F28-2.txt
BB 3: (662.3483, 1459.7685, 1655.8707, 2262.9125) -> (1265.0874, 1872.1293, 2068.2314, 2865.6519)
d2b33b91-S0-20230314-F41-2.txt
BB 3: (499.4758, 1351.2355, 1232.4022, 2127.2463) -> (1400.7537, 2295.5979, 2176.7645, 3028.5242)
d2c4c4fc-S0-20230307-F1-1.txt
BB 3: (1398.0000, 1518.0000, 2052.0000, 2274.0000) -> (1254.0000, 1476.0000, 2010.0000, 2130.0000)
d4007161-S0-20230308-F40-3.txt
BB 3: (852.3662, 1410.9287, 1862.1759, 2100.1131) -> (1427.8870, 1665.8241, 2117.0713, 2675.6338)
d4311b87-S0-20230314-F24-2.txt
BB 3: (776.3591, 1416.3553, 1748.1652, 2235.7793) -> (1292.2207, 1779.8348, 2111.6447, 2751.6409)
d45d3cca-S0-20230317-F11-2.txt
BB 3: (1699.3034, 1123.3163, 2660.2513, 1839.6339) -> (1688.3661, 867.7485, 2404.6837, 1828.6965)
d4751316-S0-20230316-F10-2.txt
BB 3: (1737.3070, 2322.6057, 2687.3968, 3104.0431) -> (423.9570, 840.6033, 1205.3943, 1790.6930)


BB 3: (1281.2639, 1145.0229, 2296.5027, 1779.9408) -> (1748.0592, 1231.4973, 2382.9772, 2246.7361)
e6f86c89-S0-20230318-F35-3.txt
BB 3: (748.6256, 545.3782, 3246.7568, 2547.8116) -> (980.1885, 281.2432, 2982.6218, 2779.3745)
e7b7f6c1-S0-20230307-F3-3.txt
BB 3: (1140.0000, 1698.0000, 1752.0000, 2442.0000) -> (1086.0000, 1776.0000, 1830.0000, 2388.0000)
e7eed599-S0-20230308-F9-1.txt
BB 3: (542.9084, 2176.0861, 1471.2819, 2843.5639) -> (684.4360, 2056.7183, 1351.9138, 2985.0916)
e843153f-S0-20230308-F11-2.txt
BB 3: (1585.2926, 1150.4495, 2633.1059, 2094.6864) -> (1433.3135, 894.8940, 2377.5504, 1942.7074)
e8e73bd6-S0-20230323-F37-1.txt
BB 3: (1615.2412, 1803.0040, 2140.0928, 2421.6420) -> (1106.3579, 1387.9072, 1724.9961, 1912.7588)
ea7d92fe-S0-20230313-F9-2.txt
BB 3: (716.6391, 2138.0996, 1655.8707, 2886.9771) -> (641.0229, 1872.1293, 1389.9004, 2811.3608)
ea7eb852-S0-20230315-F9-2.txt
BB 3: (754.6427, 1535.7416, 1563.5763, 2203.2194) -> (1324.7805, 1964.4237, 1992.2584, 2773.3574)
ea8c8